## Netflix data

In [1]:
# Import necessary libraries
import pandas as pd
import glob
from ydata_profiling import ProfileReport
import numpy as np
import json
import duckdb
import string
import re

In [2]:
# Read the Netflix file to a df
df = pd.read_csv('netflix_titles.csv', index_col=1)

# Filter on type Movie
df = df[(df.index == 'Movie')]
df
# df = df.drop(columns=['Unnamed: 0'])
df = df.replace('<NA>', np.nan)


In [3]:
# Perform EDA report on imdb reviews
profile = ProfileReport(df, title="Report_NetflixData")
profile

# EDA remarks
# director has 188 (3.1%) missing values
# cast has 475 (7.7%) missing values
# country has 440 (7.2%) missing values

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
df.columns

Index(['show_id', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [5]:
df.dtypes

show_id         object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [6]:
# Convert the datatype of columns
df = df.astype({'show_id': 'string', 'title': 'string', 'director': 'string', 'cast': 'string',
                'country': 'string', 'date_added': 'datetime64[ns]', 'release_year': 'datetime64[ns]', 
                'rating': 'string', 'duration': 'string', 'listed_in': 'string', 'description' : 'string'})

In [7]:
df.dtypes

show_id                 string
title                   string
director                string
cast                    string
country                 string
date_added      datetime64[ns]
release_year    datetime64[ns]
rating                  string
duration                string
listed_in               string
description             string
dtype: object

In [8]:
def my_map(function_to_apply, list_of_inputs):
    """
    Traverses a list, apply some function to each of the elements and collects the output.
    """
    
    new_list = []
    for i in list_of_inputs:
        new_list.append(function_to_apply(i))
        
    return new_list

In [9]:
def convert_word(word):
    """
    Strips a word from all punctuation, whitespace, and digits. Then converts the word into all lower case
    and convert all characters with accents to non-accent letters.
    """
    word = re.sub(r'[^\w\s]','',word).lower()
    word = re.sub(r'[àáâãäå]', 'a', word)
    word = re.sub(r'[èéêë]', 'e', word)
    word = re.sub(r'[ìíîï]', 'i', word)
    word = re.sub(r'[òóôõö]', 'o', word)
    word = re.sub(r'[ùúûü]', 'u', word)
    
    return word

In [10]:
def map_cleaned_strings(dfname, columnname):
    """
    Maps new values in a df-column. From a new created dictionary, with as Key the 'old' string value and 
    as Value the 'new'-string value. The 'new'-string values are extracted with the function convert_word().
    """
    dict_old_new = {}
    for i in dfname[columnname]:
        if type(i) != str:
            continue
        else:
            dict_old_new[i] = convert_word(i)

    dfname[columnname] = dfname[columnname].map(dict_old_new)
    
    return df[columnname]

In [11]:
#call function map_cleaned_strings to apply in the dataframe on the columns title, director, cast, country
map_cleaned_strings(df, 'title')
map_cleaned_strings(df, 'director')
map_cleaned_strings(df, 'cast')
map_cleaned_strings(df, 'country')

type
Movie                                        united states
Movie                                                  NaN
Movie    united states ghana burkina faso united kingdo...
Movie                                        united states
Movie                               germany czech republic
                               ...                        
Movie                          united arab emirates jordan
Movie                                        united states
Movie                                        united states
Movie                                        united states
Movie                                                india
Name: country, Length: 6131, dtype: object

In [12]:
df

show_id                            title                      director  \
type                                                                           
Movie      s1             dick johnson is dead               kirsten johnson   
Movie      s7  my little pony a new generation  robert cullen jose luis ucha   
Movie      s8                          sankofa                  haile gerima   
Movie     s10                     the starling                theodore melfi   
Movie     s13                     je suis karl           christian schwochow   
...       ...                              ...                           ...   
Movie   s8802                          zinzana               majid al ansari   
Movie   s8803                           zodiac                 david fincher   
Movie   s8805                       zombieland               ruben fleischer   
Movie   s8806                             zoom                  peter hewitt   
Movie   s8807                           zubaan                   mozez singh   

                                                    cast  \
type                                                       
Movie                                                NaN   
Movie  vanessa hudgens kimiko glenn james marsden sof...   
Movie  kofi ghanaba oyafunmike ogunlano alexandra dua...   
Movie  melissa mccarthy chris odowd kevin kline timot...   
Movie  luna wedler jannis niewohner milan peschel edi...   
...                                                  ...   
Movie  ali suliman saleh bakri yasa ali aljabri manso...   
Movie  mark ruffalo jake gyllenhaal robert downey jr ...   
Movie  jesse eisenberg woody harrelson emma stone abi...   
Movie  tim allen courteney cox chevy chase kate mara ...   
Movie  vicky kaushal sarahjane dias raaghav chanana m...   

                                                 country date_added  \
type                                                                  
Movie                                      united states 2021-09-25   
Movie                                                NaN 2021-09-24   
Movie  united states ghana burkina faso united kingdo... 2021-09-24   
Movie                                      united states 2021-09-24   
Movie                             germany czech republic 2021-09-23   
...                                                  ...        ...   
Movie                        united arab emirates jordan 2016-03-09   
Movie                                      united states 2019-11-20   
Movie                                      united states 2019-11-01   
Movie                                      united states 2020-01-11   
Movie                                              india 2019-03-02   

                       release_year rating duration  \
type                                                  
Movie 1970-01-01 00:00:00.000002020  PG-13   90 min   
Movie 1970-01-01 00:00:00.000002021     PG   91 min   
Movie 1970-01-01 00:00:00.000001993  TV-MA  125 min   
Movie 1970-01-01 00:00:00.000002021  PG-13  104 min   
Movie 1970-01-01 00:00:00.000002021  TV-MA  127 min   
...                             ...    ...      ...   
Movie 1970-01-01 00:00:00.000002015  TV-MA   96 min   
Movie 1970-01-01 00:00:00.000002007      R  158 min   
Movie 1970-01-01 00:00:00.000002009      R   88 min   
Movie 1970-01-01 00:00:00.000002006     PG   88 min   
Movie 1970-01-01 00:00:00.000002015  TV-14  111 min   

                                              listed_in  \
type                                                      
Movie                                     Documentaries   
Movie                          Children & Family Movies   
Movie  Dramas, Independent Movies, International Movies   
Movie                                  Comedies, Dramas   
Movie                      Dramas, International Movies   
...                                                 ...   
Movie           Dramas, International Movies, Thrillers   
Movie        